# Pregunta 2: Análisis de Opiniones sobre Películas

## a) 

In [1]:
import urllib
import pandas as pd

#train_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.train"
#test_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.dev"
#train_data_f = urllib.urlretrieve(train_data_url, "train_data.csv")
#test_data_f = urllib.urlretrieve(test_data_url, "test_data.csv")

ftr = open("train_data.csv", "r")
fts = open("test_data.csv", "r")
rows = [line.split(" ",1) for line in ftr.readlines()]
train_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
train_df['Sentiment'] = pd.to_numeric(train_df['Sentiment'])
rows = [line.split(" ",1) for line in fts.readlines()]
test_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
test_df['Sentiment'] = pd.to_numeric(test_df['Sentiment'])
train_df.shape
test_df.shape

(3554, 2)

In [2]:
from IPython.display import HTML, display

n_neg_train = train_df[train_df.Sentiment == -1].count().Sentiment
n_pos_train = train_df[train_df.Sentiment == 1].count().Sentiment

n_neg_test = test_df[test_df.Sentiment == -1].count().Sentiment
n_pos_test = test_df[test_df.Sentiment == 1].count().Sentiment

data = [['','Train','Test'],
        ['Positivo',n_pos_train,n_pos_test],
        ['Negativo',n_neg_train,n_neg_test],
        ['Total',n_pos_train+n_neg_train,n_pos_test+n_neg_test]
       ]

display(HTML(
        '<table><tr>{}</tr></table>'.format(
            '</tr><tr>'.join(
                '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
    ))

,Train,Test
Positivo,1770,1751
Negativo,1784,1803
Total,3554,3554


## b) 

In [17]:
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from nltk.stem.porter import PorterStemmer

def word_extractor(text):
    stemmer = PorterStemmer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ stemmer.stem(word.lower()) \
                  for word in word_tokenize(text.decode('utf-8', 'ignore')) ]
    
    for word in wordtokens:
        if word not in commonwords:
            words+=" "+word

    return words

In [ ]:
quitar el stem y comparar

## c) 

In [12]:
def word_extractor2(text):
    wordlemmatizer = WordNetLemmatizer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ wordlemmatizer.lemmatize(word.lower()) \
                  for word in word_tokenize(text.decode('utf-8','ignore')) ]
    for word in wordtokens:
        if word not in commonwords:
            words+=" "+word
    
    return words

In [ ]:
en wikipedia se explica bien la diferencia

In [18]:
print word_extractor("I love to eat cake")
print word_extractor("I love eating cake")
print word_extractor("I loved eating the cake")
print word_extractor("I do not love eating cake")
print word_extractor("I don't love eating cake")
print 
print word_extractor2("I love to eat cake")
print word_extractor2("I love eating cake")
print word_extractor2("I loved eating the cake")
print word_extractor2("I do not love eating cake")
print word_extractor2("I don't love eating cake")

 love eat cake
 love eat cake
 love eat cake
 love eat cake
 n't love eat cake

 love eat cake
 love eating cake
 loved eating cake
 love eating cake
 n't love eating cake


## d) 

In [32]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

texts_train = [word_extractor(text) for text in train_df.Text]
texts_test = [word_extractor(text) for text in test_df.Text]

vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train))

features_train = vectorizer.transform(texts_train)
features_test = vectorizer.transform(texts_test)

labels_train = np.asarray((train_df.Sentiment.astype(float)+1)/2.0)
labels_test = np.asarray((test_df.Sentiment.astype(float)+1)/2.0)

vocab = vectorizer.get_feature_names()
dist=list(np.array(features_train.sum(axis=0)).reshape(-1,))

for tag, count in zip(vocab, dist):
    print count, tag

6 10
4 100
2 101
1 105
2 10th
4 11
2 110
1 11th
1 12
2 13
1 13th
1 14
1 140
1 146
3 15
1 16
1 163
2 170
1 18th
4 19
2 1915
1 1934
1 1938
1 1940
3 1950
1 1954
1 1955
1 1958
1 1959
2 1960
1 1967
1 1972
3 1975
1 1978
1 1979
2 1980
1 1991
1 1992
2 1995
3 19th
4 20
2 2000
7 2002
4 20th
2 21st
1 22
1 24
3 25
3 30
1 300
2 3000
1 37
1 3d
6 40
1 400
1 401
1 451
1 48
1 49
4 4ever
3 50
2 51
1 53
1 5th
1 60
1 60s
1 65
1 65th
4 70
1 71
1 77
1 78
1 79
7 80
1 800
2 84
2 85
1 88
1 8th
14 90
1 93
1 94
2 95
1 96
1 97
2 99
1 aaliyah
2 abandon
1 abandono
1 abbass
1 abbrevi
1 abc
2 abel
1 abhorr
1 abid
11 abil
6 abl
3 abli
1 aboul
14 abov
3 above
1 abraham
1 abram
1 abras
2 abroad
1 abruptli
9 absolut
12 absorb
1 absorpt
5 abstract
6 absurd
3 absurdist
1 absurdli
1 aburrido
4 abus
1 abysm
1 acabamo
1 academ
2 academi
7 accent
1 accentu
3 accept
10 access
1 accid
2 accident
3 accompani
6 accomplish
1 accord
9 account
1 accumul
3 accur
1 accus
1 acerta
1 ach
13 achiev
1 achil
4 achingli
1 achiv
1 achronolog


## e) 

In [35]:
from sklearn.metrics import classification_report

def score_the_model(model,x,y,xt,yt,text):
    acc_tr = model.score(x,y)
    acc_test = model.score(xt[:-1],yt[:-1])
    print "Training Accuracy %s: %f"%(text,acc_tr)
    print "Test Accuracy %s: %f"%(text,acc_test)
    print "Detailed Analysis Testing Results ..."
    print(classification_report(yt, model.predict(xt), target_names=['+','-']))

## f) 

In [45]:
from sklearn.naive_bayes import BernoulliNB
import random

def do_NAIVE_BAYES(x,y,xt,yt):
    model = BernoulliNB()
    model = model.fit(x, y)
    score_the_model(model,x,y,xt,yt,"BernoulliNB")
    return model

model=do_NAIVE_BAYES(features_train,labels_train,features_test,labels_test)
test_pred = model.predict_proba(features_test)
spl = random.sample(xrange(len(test_pred)), 15)

for text, sentiment in zip(test_df.Text[spl], test_pred[spl]):
    print sentiment, text

Training Accuracy BernoulliNB: 0.942881
Test Accuracy BernoulliNB: 0.747819
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.74      0.75      1803
          -       0.74      0.75      0.75      1751

avg / total       0.75      0.75      0.75      3554

[ 0.22798674  0.77201326] if you're not fans of the adventues of steve and terri , you should avoid this like the dreaded king brown snake . personally , i'd rather watch them on the animal planet .

[ 0.36107799  0.63892201] in both the writing and cutting , it does not achieve the kind of dramatic unity that transports you . you end up simply admiring this bit or that , this performance or that .

[ 0.76792616  0.23207384] soul is what's lacking in every character in this movie and , subsequently , the movie itself .

[ 0.72807661  0.27192339] it's difficult to imagine that a more confused , less interesting and more sloppily made film could possibly come down 

## g) 

In [47]:
from sklearn.naive_bayes import MultinomialNB
import random

def do_MULTINOMIAL(x,y,xt,yt):
    model = MultinomialNB()
    model = model.fit(x, y)
    score_the_model(model,x,y,xt,yt,"MULTINOMIAL")
    return model

model=do_MULTINOMIAL(features_train,labels_train,features_test,labels_test)
test_pred = model.predict_proba(features_test)
spl = random.sample(xrange(len(test_pred)), 15)
for text, sentiment in zip(test_df.Text[spl], test_pred[spl]):
    print sentiment, text

Training Accuracy MULTINOMIAL: 0.942319
Test Accuracy MULTINOMIAL: 0.749789
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.75      0.75      1803
          -       0.74      0.75      0.75      1751

avg / total       0.75      0.75      0.75      3554

[ 0.21222454  0.78777546] [cho's face is] an amazing slapstick instrument , creating a scrapbook of living mug shots .

[ 0.62736946  0.37263054] as it turns out , you can go home again .

[ 0.10151959  0.89848041] 13 conversations may be a bit too enigmatic and overly ambitious to be fully successful , but sprecher and her screenwriting partner and sister , karen sprecher , don't seem ever to run out of ideas .

[ 0.92914794  0.07085206] it's drained of life in an attempt to be sober and educational , and yet it's so devoid of realism that its lack of whistles and bells just makes it obnoxious and stiff .

[ 0.42736134  0.57263866] finally , a genre movie that d

## h) 

In [48]:
from sklearn.linear_model import LogisticRegression

def do_LOGIT(x,y,xt,yt):
    start_t = time.time()
    Cs = [0.01,0.1,10,100,1000]
    for C in Cs:
        print "Usando C= %f"%C
        model = LogisticRegression(penalty='l2',C=C)
        model = model.fit(x, y)
        score_the_model(model,x,y,xt,yt,"LOGISTIC")

do_LOGIT(features_train,labels_train,features_test,labels_test)

Usando C= 0.010000
Training Accuracy LOGISTIC: 0.782217
Test Accuracy LOGISTIC: 0.690684
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.68      0.73      0.70      1803
          -       0.70      0.65      0.68      1751

avg / total       0.69      0.69      0.69      3554

Usando C= 0.100000
Training Accuracy LOGISTIC: 0.880135
Test Accuracy LOGISTIC: 0.731213
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.73      0.75      0.74      1803
          -       0.73      0.71      0.72      1751

avg / total       0.73      0.73      0.73      3554

Usando C= 10.000000
Training Accuracy LOGISTIC: 0.999719
Test Accuracy LOGISTIC: 0.725303
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.73      0.72      0.73      1803
          -       0.72      0.73      0.72      1751

avg / total       0.73     

## i) 

In [ ]:
from sklearn.svm import LinearSVC

def do_SVM(x,y,xt,yt):
    Cs = [0.01,0.1,10,100,1000]
    for C in Cs:
        print "El valor de C que se esta probando: %f"%C
        model = LinearSVC(C=C)
      
    model = model.fit(x, y)
        score_the_model(model,x,y,xt,yt,"SVM")

do_SVM(features_train,labels_train,features_test,labels_test)
###

## j) 